# README
Mean Average Precision(MAP@K) を計算する

## 参考
- [scikit-learnでMean Average Precisionを計算しようと思ったら混乱した話 - 唯物是真 @Scaled_Wurm](http://sucrose.hatenablog.com/entry/2017/02/26/224559)
- [レコメンドつれづれ　～第3回 レコメンド精度の評価方法を学ぶ～ - Platinum Data Blog by BrainPad](http://blog.brainpad.co.jp/entry/2017/08/25/140000)
- [sklearn.metrics.label_ranking_average_precision_score — scikit-learn 0.21.2 documentation](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.label_ranking_average_precision_score.html)


In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import (average_precision_score, confusion_matrix,
                             label_ranking_average_precision_score,
                             precision_recall_curve, precision_score,
                             recall_score)

# 例その1
- [scikit-learnでMean Average Precisionを計算しようと思ったら混乱した話 - 唯物是真 @Scaled_Wurm](http://sucrose.hatenablog.com/entry/2017/02/26/224559)


## Dataの準備

In [2]:
data = pd.DataFrame()

data['y_true'] = [0, 0, 0, 0, 0, 1, 1, 1, 0, 0, ]
data['y_pred'] = [0, 1, 0, 1, 0, 0, 1, 0, 1, 0, ]
data['y_prob'] = [0.30, 0.80, 0.45, 0.95, 0.20,
                  0.01, 0.98, 0.30, 0.94, 0.02]

data

,y_true,y_pred,y_prob
0,0,0,0.30
1,0,1,0.80
2,0,0,0.45
3,0,1,0.95
4,0,0,0.20
5,1,0,0.01
6,1,1,0.98
7,1,0,0.30
8,0,1,0.94
9,0,0,0.02


## Precisionの計算

In [3]:
# 正例と予測したもの
data.query('y_pred == 1')

,y_true,y_pred,y_prob
1,0,1,0.80
3,0,1,0.95
6,1,1,0.98
8,0,1,0.94


In [4]:
# Precision := 1と予測したもののうち、 正しく1と予測できた割合
data.query(
    'y_pred == 1 and y_true == 1').shape[0] / data.query('y_pred == 1').shape[0]

0.25

In [5]:
# sklearnでも確認
precision_score(data['y_true'], data['y_pred'])

0.25

## Recallの計算

In [6]:
# データ内の正例
data.query('y_true == 1')

,y_true,y_pred,y_prob
5,1,0,0.01
6,1,1,0.98
7,1,0,0.30


In [7]:
# Recall := データ内の正例のうち、正例と予測したものの割合
data.query(
    'y_true == 1 and y_pred == 1').shape[0] / data.query('y_true == 1').shape[0]

0.3333333333333333

In [8]:
# sklearnでも確認
recall_score(data['y_true'], data['y_pred'])

0.3333333333333333

## ConfusionMatrixも一応みておく

In [9]:
confusion_matrix(y_true=data['y_true'],
                 y_pred=data['y_pred'])

array([[4, 3],
       [2, 1]])

## Average Precision の グラフ
- 適合率・再現率カーブとも言うみたいだ
    - 『Python機械学習プログラミング』 p.284

In [10]:
precision, recall, thresholds = precision_recall_curve(y_true=data['y_true'],
                                                       probas_pred=data['y_prob'])

In [11]:
# これあってんの？
plt.plot(precision, recall)
plt.xlabel('Precision')
plt.ylabel('Recall')
plt.xlim(0, 1)
plt.ylim(0, 1)

NameError: name 'plt' is not defined

In [ ]:
average_precision_score(y_true=data['y_true'],
                        y_score=data['y_prob'],
                        average='samples')

## Average Precision の計算

## データの準備

In [ ]:
# 上位ランクは DFのindex で代用している！！！！！！
data = pd.DataFrame()

data['y_true'] = [1, 0, 1, 0]
data['y_score'] = [1.0, 0.8, 0.6, 0.4]  # average_precision_score() を使うなら必要
data['y_pred'] = [1, 1, 1, 0]

data

### 素で計算

In [ ]:
# AveragePrecision := 「データ内の正例」 のうち、
#                     (k番目までの出力で計算したPrecision) * I(k)
# I(k) : = 出力をスコア上位順に並べた時にk番目の出力が正例なら1、負例なら0になる関数

In [ ]:
def ap(k: int, data):
    """データ内の正例のうち、k番目までの出力で計算したPrecision"""
    num_y_true = data.iloc[:k].query('y_pred == 1').shape[0]
    precision_k = data.iloc[:k].query('y_pred == 1 and y_true == 1').shape[0]

    return precision_k / num_y_true


assert ap(1, data) == precision_score(
    data.iloc[:1]['y_true'], data.iloc[:1]['y_pred'])
assert ap(2, data) == precision_score(
    data.iloc[:2]['y_true'], data.iloc[:2]['y_pred'])
assert ap(3, data) == precision_score(
    data.iloc[:3]['y_true'], data.iloc[:3]['y_pred'])
assert ap(4, data) == precision_score(
    data.iloc[:4]['y_true'], data.iloc[:4]['y_pred'])

In [ ]:
def I(k: int, data: pd.DataFrame):
    """出力をスコア上位順に並べた時にk番目の出力が正例なら1、負例なら0になる関数"""
    return int(data.iloc[k-1]['y_true'] == 1)


I(1, data), I(2, data), I(3, data), I(4, data)

In [ ]:
ap_k1 = ap(1, data) * I(1, data)
ap_k2 = ap(2, data) * I(2, data)
ap_k3 = ap(3, data) * I(3, data)
ap_k4 = ap(4, data) * I(4, data)

ap_values = np.array([ap_k1, ap_k2, ap_k3, ap_k4])

ap_values

In [ ]:
map_k4 = ap_values.sum() / (ap_values != 0).sum()

map_k4

In [ ]:
#  sklearn.metrics.average_precision_score と 一致した！！！
average_precision_score(
    y_true=data['y_true'],
    y_score=data['y_score'],
    pos_label=1  # 「1」を正例とする
)

### sklearn.metrics.average_precision_score の 動き

In [ ]:
s1 = average_precision_score(y_true=[1, 0, 1, 0],
                             y_score=[1, 0.8, 0.6, 0.4],
                             average='samples')

s2 = average_precision_score(y_true=[1, 0, 1, 0],
                             y_score=[1, 0.9, 0.6, 0.4],
                             average='samples')

s3 = average_precision_score(y_true=[1, 0, 1, 0],
                             y_score=[1, 0.7, 0.3, 0.1],
                             average='samples')

# 予測「確率」が異なっても結果は変わらないよ
s1, s2, s3

In [ ]:
# 予測「確率」じゃなくなると挙動が変わる！？
s4 = average_precision_score(y_true=[1, 0, 1, 0],
                             y_score=[1, 0, 0, 1],
                             average='samples')

s4

## average_precision_score と  label_ranking_average_precision_score は同じ挙動になったらしい(sklearn >= 0.19.1)

In [ ]:
average_precision_score([1, 0, 1, 0], [1, 0.8, 0.6, 0.4], average='samples')

In [ ]:
label_ranking_average_precision_score([[1, 0, 1, 0]], [[1, 0.8, 0.6, 0.4]])

# 例その2:
- [レコメンドつれづれ　～第3回 レコメンド精度の評価方法を学ぶ～ - Platinum Data Blog by BrainPad](http://blog.brainpad.co.jp/entry/2017/08/25/140000)
    - 記事の結構下の方を参照

In [ ]:
# 「yの予測」 と 「yの予測確率」 は なくても計算できるんだぞ！！！！
# ただし、 sklearn.metrics.average_precision_score を使うなら必要になるよ
# まあ、モデルがあれば、予測も予測確率も計算できるからあんまり関係ない
data = pd.DataFrame()

data['rank'] = [1, 2, 3, 4, 5, 6]
data['y_true'] = [0, 1, 0, 0, 1, 1]


data

In [ ]:
k = 6

# 上位K番目のIndexを取得
rank_list_y_true = []

for idx, d in data.iterrows():
    rank = d['rank']
    y_true = d['y_true']

    if rank > k:
        continue

    if y_true == 1:
        rank_list_y_true.append(idx + 1)

# MAP@Kを計算する
ap_values = []

for tmp_k in rank_list_y_true:
    partial_data = data.iloc[:tmp_k]  # 上位K番目までのデータ

    num_y_true = partial_data.query('y_true == 1').shape[0]
    tmp_ap = num_y_true / tmp_k

    ap_values.append(tmp_ap)

ap_values = np.array(ap_values)

print(f'MAP@{k} = {ap_values.mean():.2f}')

# おわりに
- 計算方法はだいたいわかった
- もう何度かやらないと記憶は出来ないけど、まあこのNotebookを見れば行けるはず
- 上位RankとDFのIndexが少々ややこしいね